### 有序表
* 现实使用中需要进行类似14～27这种范围的查询，相比哈希表，有序表更实用
* 增删改查(logN)
* 包括AVL、SB树、红黑树，只是平衡限制不同。AVL的最严

#### 左旋

In [ ]:
'''

    A                             B
T       B        左旋A==>      A       D
      C   D                 T    C

A变A右子树的左子树
A右子树的左子树变A的右子树
'''

#### 右旋

In [ ]:
'''
      A                             B
  B       T      右旋A==>        C       A
C   D                                 D    T
A变左子树的右子树
A左子树的右子树变A的左子树
'''

### AVL
任何一个节点，|左高-右高|<2

#### 删除一个节点
* 情况1: 无左无右，直接删
* 情况2: 有左无右 或 有右无左， 用唯有的子树代替当前节点
* 情况3: 有左有右，用右树的左边界最后一个节点代替。右树的左边界最后一个节点一定没有左孩子，如果它有右孩子，放到它的位置(情况2)

#### 破坏平衡性的四种类型
1. 位置说明：父 -> 子 -> 孙；调整代价为O(1)
2. 加入节点：沿途往上节点每个都检查，尝试调整。 所以代价为logN
3. 删除节点：无左无右，删除后，沿途往上节点每个都检查； 有右无左或有左无右，替代节点位置沿途往上；既有左又有右，从左边界最后一个节点沿途往上查

    * LL型：父右旋
    * LR型：子左旋，父右旋(即想办法将孙换到父位置)   判断：左树-右树高度>=2，且左树的右树比 左树的左树 高
    * RR型：父左旋
    * RL型：子右旋，父左旋
    * 既是LL型又是LR型：按照LL型调整，直接父右旋   （因为按LL调总有效，而LR不一定）


#### 代码实现

In [ ]:
class AVLNode:
    def __init__(self, key, value, l=None, r=None, h=1):
        self.key = key
        self.value = value
        self.l = l
        self.r = r
        self.h = h
        
class AVLTreeMap:
    def __init__(self, root=None, size=0):
        self.root = root
        self.size = size
        
    def rightRotate(self, cur):
        left = cur.l
        cur.l = left.r 
        
        left.r = cur
        
        cur_left_h = 0 if cur.l is None else cur.l.h
        cur_right_h = 0 if cur.r is None else cur.r.h
        cur.h = max(cur_left_h, cur_right_h) + 1
        
        l_left_h = 0 if left.l is None else left.l.h
        l_right_h = 0 if left.r is None else left.r.h
        left.h = max(l_left_h, l_right_h) + 1
        return left
    
    def leftRotate(self, cur):
        right = cur.r
        cur.r = right.l
        right.l = cur
        
        cur_left_h = 0 if cur.l is None else cur.l.h
        cur_right_h = 0 if cur.r is None else cur.r.h
        cur.h = max(cur_left_h, cur_right_h) + 1
        
        r_left_h = if right.l is None else right.l.h
        r_right_h = 0 if right.r is None else right.r.h
        right.h = max(r_left_h, r_right_h) + 1
        return right
    
    def add(self, cur, key, value): #不接受重复key！
        if cur is None:
            return AVLNode(key, value)
        else:
            if key < cur.k:
                cur.l = self.add(cur.l, key, value) #有可能换头
            else:
                cur.r = self.add(cur.r, key, value)
            left_h = 0 if cur.l is None else cur.l.h
            right_h = 0 if cur.r is None else cur.r.h
            cur.h = max(left_h, right_h) + 1
            return self.maintain(cur) #调整平衡
        
    def delete(self, cur, key): #需要保证key一定出现在当前树上！
        if key < cur.k:
            cur.l = self.delete(cur.l, key)
        elif key > cur.k:
            cur.r = self.delete(cur.r, key)
        else:
            if cur.l is None and cur.r is None:
                cur = None
            elif cur.l is None and cur.r is not None:
                cur = cur.r
            elif cur.l is not None and cur.r is None:
                cur = cur.l
            else:
                des = cur.r
                while des.l is not None:
                    des = des.l
                cur.r = self.delete(cur.r, des.k)
                des.l = cur.l
                des.r = cur.r
                cur = des
        if cur is not None:
            left_h = 0 if cur.l is None else cur.l.h
            right_h = 0 if cur.r is None else cur.r.h
            cur.h = max(left_h, right_h) + 1
        return self.maintain(cur)
    
    def maintain(cur):
        if cur is None:
            return None
        
        left_h = 0 if cur.l is None else cur.l.h
        right_h = 0 if cur.r is None else cur.r.h
        
        if abs(left_h - right_h) > 1:
            if left_h > right_h:
                left_left_h = cur.l.l.h if cur.l is not None and cur.l.l is not None else 0
                left_right_h = cur.l.r.h if cur.l is not None and cur.l.r is not None else 0
                if left_left_h >= left_right_h:
                    cur = self.rightRotate(cur)
                else:
                    cur.l = self.leftRotate(cur.l)
                    cur = self.rightRotate(cur)
            else:
                right_left_h = cur.r.l.h if cur.r is not None and cur.r.l is not None else 0
                right_right_h = cur.r.r.h if cur.l is not None and cur.r.r is not None else 0
                if right_right_h >= right_left_h:
                    cur = self.leftRotate(cur)
                else:
                    cur.r = self.rightRotate(cur.r)
                    cur = self.leftRotate(cur)
            return cur
                    
        
                    
                
                
        